In [ ]:
import h5py
import numpy as np
import os

In [ ]:
metric_dict = {
    'layer_mse': 'MSE',
    'ami_obj': 'AMI',
    'count': 'OCA',
    'order': 'OOA',
}

def convert(scores_mean, scores_std, key, use_ooa):
    text = '{}:  '.format(metric_dict[key])
    if key == 'layer_mse':
        text += '{:.2e} {} {:.0e}'.format(scores_mean[key], u'\u00B1', scores_std[key], key)
    elif key == 'order' and not use_ooa:
        text += 'N/A'
    else:
        text += '{:.3f} {} {:.0e}'.format(scores_mean[key], u'\u00B1', scores_std[key], key)
    return text

def print_scores_single(phase):
    folder = 'outs_single'
    for name in ['mnist', 'abstract']:
        print(name)
        path_result = os.path.join(folder, '{}_1.0_10.0'.format(name), '{}.h5'.format(phase))
        with h5py.File(path_result, 'r') as f:
            scores_mean = {key: np.mean(f['metric'][key]) for key in f['metric']}
            scores_std = {key: np.std(f['metric'][key]) for key in f['metric']}
        scores_list = [convert(scores_mean, scores_std, key, False) for key in ['layer_mse', 'count']]
        print((' ' * 8).join(scores_list))
    return

def print_scores_complex(phase):
    for name in ['mnist', 'abstract']:
        for occl in [0, 1]:
            for mode in ['direct', 'adi']:
                print(name, occl, mode)
                folder = 'outs_{}'.format(mode)
                suffix = '1.0' if mode == 'direct' else '1.0_1.0'
                path_result = os.path.join(folder, '{}_{}_{}'.format(name, occl, suffix), '{}.h5'.format(phase))
                with h5py.File(path_result, 'r') as f:
                    scores_mean = {key: np.mean(f['metric'][key]) for key in f['metric']}
                    scores_std = {key: np.std(f['metric'][key]) for key in f['metric']}
                scores_list = [convert(scores_mean, scores_std, key, name == 'abstract')
                               for key in ['layer_mse', 'ami_obj', 'count', 'order']]
                print((' ' * 8).join(scores_list))
    return

In [ ]:
# Performance of Supervised Learning
print_scores_single('test')

In [ ]:
# Effectiveness of Acquired Impressions (tested on images containing 2 ~ 4 objects)
print_scores_complex('test')

In [ ]:
# Effectiveness of Acquired Impressions (tested on images containing 5 ~ 6 objects)
print_scores_complex('general')